In [ ]:
!mkdir .kaggle

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

X = np.load("/content/train.npy", allow_pickle=True)
Y = np.load("/content/train_labels.npy", allow_pickle=True)


In [ ]:
for x in X:
  total_datapoints += x.shape[0]
print(total_datapoints)
print(X[0].shape)
print(Y[0].shape)

27329537
(1406, 13)
(1406,)


In [ ]:
from torch.utils.data import Dataset, DataLoader

CONTEXT_SIZE = 24

class MyDataset(Dataset):
  def __init__(self, X, Y):
    super(MyDataset, self).__init__()
    self.X = []
    self.map = [0] * total_datapoints
    current_lower = 0
    convert_Y = np.array([])
    for index, utterance in np.ndenumerate(X):
      size = utterance.shape[0]
      padded_utterance = np.pad(utterance,((CONTEXT_SIZE,), (0,)), 'constant')
      self.X.append(padded_utterance)
      self.map[current_lower:current_lower+size] = [(index[0], k+CONTEXT_SIZE) for k in range(size)]
      current_lower += size
      convert_Y = np.concatenate((convert_Y, Y[index]))

    print("X length is ", len(self.X))
    self.Y = torch.from_numpy(convert_Y)
    print("Y shape is ", self.Y.size())
      


  def __len__(self):
    return list(self.Y.size())[0]

  def __getitem__(self, index):
    utter_idx, frame_idx = self.map[index]
    utter = self.X[utter_idx]       
    X = torch.from_numpy(utter[frame_idx-CONTEXT_SIZE:frame_idx+CONTEXT_SIZE+1].flatten())
    X = X.type(torch.FloatTensor)
    Y = self.Y[index]
    Y = Y.type(torch.LongTensor)
    return X,Y

In [ ]:
def xavier_init(params):
  for m in params:
    if isinstance(m, nn.Linear):
      nn.init.xavier_normal_(m.weight)
      nn.init.zeros_(m.bias)

def zero_init(params):
  for m in params:
    if isinstance(m, nn.Linear):
      nn.init.zeros_(m.weight)
      nn.init.zeros_(m.bias)

class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()

    self.layers = nn.Sequential(
        nn.Linear((2*CONTEXT_SIZE+1)*13,2048),
        nn.ReLU(),
        nn.BatchNorm1d(2048),
        nn.Linear(2048,1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,1024),
        nn.ReLU(),
        nn.BatchNorm1d(1024),
        nn.Linear(1024,512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Linear(512,346)
    )
  
  def init_weights(self):
    with torch.no_grad():
      xavier_init(self.modules())

  def forward(self, x):
    out = self.layers(x)
    
    return out

  def save(self, ckpt_path):
    ckpt = {
        'params': self.state_dict()
    }
    torch.save(ckpt, ckpt_path)
  
  def load(self, ckpt_path):
    ckpt = torch.load(ckpt_path)
    self.load_state_dict(ckpt['params'], strict=True)

In [ ]:
cuda = torch.cuda.is_available()

num_of_workers = 0 if cuda else 0
print("Init started")
train_dataset = MyDataset(X, Y)
del X
del Y
print("Init completed")
train_loader_args = dict(shuffle=True, batch_size=2048, num_workers=num_of_workers, pin_memory=True, drop_last=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader = DataLoader(train_dataset, **train_loader_args)

model = MyModel()
device = torch.device("cuda" if cuda else "cpu")
print(device)
model.to(device)
print(model)


Init started
X length is  22002
Y shape is  torch.Size([27329537])
Init completed
cuda
MyModel(
  (layers): Sequential(
    (0): Linear(in_features=663, out_features=2048, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=1024, out_features=1024, bias=True)
    (10): ReLU()
    (11): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Linear(in_f

In [ ]:
print(len(train_dataset))

27329537


In [ ]:
import torch.optim as optim
import time

NUM_EPOCHS = 8
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma=0.1)
class AvgMeter():
    def __init__(self):
        self.qty = 0
        self.cnt = 0
    
    def update(self, increment, count):
        self.qty += increment
        self.cnt += count
    
    def get_avg(self):
        if self.cnt == 0:
            return 0
        else: 
            return self.qty/self.cnt


for epoch in range(NUM_EPOCHS):
  model.train()
  loss_meter = AvgMeter()
  accu_meter = AvgMeter()
  running_loss = 0.0
  start_time = time.time()
  for i, data in enumerate(train_loader):
    x, y = data
    optimizer.zero_grad()

    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)
    _, predicted = torch.max(output, 1)
    running_loss += loss.item()
    loss_meter.update(loss.item(), 1)
    accu_meter.update((predicted == y).sum().item(), y.size(0))

    loss.backward()
    optimizer.step()

    if i % 2000 == 1999:
      print(f"{epoch+1} epoch, {i+1} batch, loss: {running_loss/2000}")
      end_time = time.time()
      print(f"2000 batches take {end_time-start_time} seconds")
      running_loss = 0.0
      start_time = time.time()
  
  scheduler.step()  
  loss = loss_meter.get_avg()
  accu = accu_meter.get_avg()
  print(f"The loss is {loss} and the accuracy is {accu} in Epoch {epoch+1}")

print('Finished Training')
model.save("/content/ckpt.pth")
torch.cuda.empty_cache()


1 epoch, 2000 batch, loss: 1.5257254166007042
2000 batches take 209.87855529785156 seconds
1 epoch, 4000 batch, loss: 1.1565340431928635
2000 batches take 205.4843692779541 seconds
1 epoch, 6000 batch, loss: 1.043668802201748
2000 batches take 205.5911991596222 seconds
1 epoch, 8000 batch, loss: 0.9748268194794655
2000 batches take 204.3959767818451 seconds
1 epoch, 10000 batch, loss: 0.9282104606330395
2000 batches take 204.53612279891968 seconds
1 epoch, 12000 batch, loss: 0.8930591180622578
2000 batches take 203.64968061447144 seconds
The loss is 1.0648294854909182 and the accuracy is 0.680469869709701 in Epoch 1
2 epoch, 2000 batch, loss: 0.8366633911132813
2000 batches take 206.5890371799469 seconds
2 epoch, 4000 batch, loss: 0.8204503124356269
2000 batches take 203.66243314743042 seconds
2 epoch, 6000 batch, loss: 0.8068840204775334
2000 batches take 204.01112246513367 seconds
2 epoch, 8000 batch, loss: 0.7933611751496792
2000 batches take 204.70449876785278 seconds
2 epoch, 1000

In [ ]:
test = np.load("/content/test.npy", allow_pickle=True)
total_test_datapoints = 0
for i in test:
  total_test_datapoints += i.shape[0]

class MyTestDataset(Dataset):
  def __init__(self, X):
    super(MyTestDataset, self).__init__()

    self.X = []
    self.map = [0] * total_test_datapoints
    self.size = 0
    current_lower = 0
    for index, utterance in np.ndenumerate(X):
      size = utterance.shape[0]
      self.size += size
      padded_utterance = np.pad(utterance,((CONTEXT_SIZE,), (0,)), 'constant')
      self.X.append(padded_utterance)
      self.map[current_lower:current_lower+size] = [(index[0], k+CONTEXT_SIZE) for k in range(size)]
      current_lower += size

  def __len__(self):
    return self.size

  def __getitem__(self, index):
    utter_idx, frame_idx = self.map[index]
    utter = self.X[utter_idx]       
    X = torch.from_numpy(utter[frame_idx-CONTEXT_SIZE:frame_idx+CONTEXT_SIZE+1].flatten())
    X = X.type(torch.FloatTensor)
    return X


test_dataset = MyTestDataset(test)
del test
test_loader_args = dict(shuffle=False, batch_size=256, num_workers=0, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=64)
test_loader = DataLoader(test_dataset, **test_loader_args)


In [ ]:
model.eval()
pred = []
with torch.no_grad():
  for data in test_loader:
    data = data.to(device)
    outputs = model(data)
    _, predicted = torch.max(outputs.data, 1)
    pred += predicted.data.cpu().tolist()
    del predicted
    del outputs
    # torch.cuda.empty_cache()

print("Done prediction!")
df = pd.DataFrame(pred, columns=['label'])
df.to_csv("/content/submission.csv", index_label="id")

Done prediction!
